# Analysis of Layout ID

## Relevant documents

- (Python Client Repo)[https://github.com/Green-Fusion/energy-management-backend/tree/main/python_client]
- (Klemmenbelegung)[https://docs.google.com/spreadsheets/d/1nkdkx2rI6nVKgoKBgkCUtfwEwuv8kptrRUXcXtfv0NM/edit?gid=247168398#gid=247168398]
- (Hypothesis for Klemmenbelegung)[https://docs.google.com/spreadsheets/d/1TSTxMCgEvuoayzOfx1MUqlV0tiqsVTBRN8aldlnFXxA/edit?gid=0#gid=0]

# import data

In [48]:
#imports
import os
import pandas as pd
import json
import ast
from pprint import pprint

In [2]:
#build df from database exctracted data
filename='Building_device_Layout2025-07-14.csv'
extraced_date=filename[-14:-4]
filepath = os.path.join('INs', filename)
df = pd.read_csv(filepath)
#inspest column
df.columns

Index(['building_id', 'customerID', 'customer_name', 'address', 'postal_code',
       'city', 'coordinates', 'LayoutID', 'device_type', 'device_id',
       'modular_system', 'gfid'],
      dtype='object')

In [3]:
#filter out uncesseray infos
df=df[['building_id','customerID', 'customer_name','device_type', 'customer_name','address','LayoutID','modular_system']]

In [4]:
df.head()

,building_id,customerID,customer_name,device_type,customer_name,address,LayoutID,modular_system
0,756,66,HwS,RUT956,HwS,Wolzogenstr.28,gas:1--gas:2--heating-circuit:1--heating-circu...,"{'name': 'modular-system-entry', 'id': 'modula..."
1,757,66,HwS,RUT956,HwS,Hagelberger Str. 26,gas:1--gas:2--heating-circuit:1,"{'name': 'modular-system-entry', 'id': 'modula..."
2,758,66,HwS,RUT956,HwS,Hochstr. 8,gas:1--heating-circuit:1,"{'name': 'modular-system-entry', 'id': 'modula..."
3,759,66,HwS,RUT956,HwS,Planufer 82a,gas:1--heating-circuit:1,"{'name': 'modular-system-entry', 'id': 'modula..."
4,761,66,HwS,RUT956,HwS,Gabainstr. 13,gas:1--gas:2--global-separation-circuit:1--hea...,"{'name': 'modular-system-entry', 'id': 'modula..."


In [5]:
df.count()

building_id       2014
customerID        2014
customer_name     2014
device_type       1086
customer_name     2014
address           2014
LayoutID          1138
modular_system    2014
dtype: int64

# Group by LayoutID

In [6]:
unique_count = df['LayoutID'].nunique()
print(f"Number of unique LayoutID values: {unique_count}")

Number of unique LayoutID values: 233


In [7]:
# Group by LayoutID
grouped = df.groupby('LayoutID').agg(
    Occurrence=('LayoutID', 'count'),
    buildingIDs=('building_id', list)
).reset_index()

# Add a simple index column starting from 0
grouped = grouped.sort_values(by='Occurrence', ascending=False).reset_index(drop=True)
grouped['KBn']=grouped.index+1
#grouped['KBn'] = grouped['KBn'].fillna(-1).astype(int)

In [8]:
# Show result
grouped.head(20)

,LayoutID,Occurrence,buildingIDs,KBn
0,gas:1--heating-circuit:1--warm-water:1,129,"[768, 769, 770, 778, 787, 1924, 1926, 1927, 46...",1
1,district-heating:1--heating-circuit:1--warm-wa...,84,"[774, 1909, 1910, 1911, 1912, 1913, 1314, 1315...",2
2,gas:1--heating-circuit:1,77,"[758, 759, 499, 1525, 1649, 1654, 1657, 1791, ...",3
3,gas:1--heat-exchanger:1--heating-circuit:1--wa...,46,"[1925, 1931, 1781, 1509, 1526, 1527, 1531, 153...",4
4,local-heating-station:1--heating-circuit:1--wa...,45,"[509, 790, 1005, 1628, 1629, 1630, 1631, 672, ...",5
5,gas:1--heat-exchanger:1--heating-circuit:1,40,"[1524, 1795, 1560, 1638, 1639, 604, 617, 623, ...",6
6,gas:1--heating-circuit:1--heating-circuit:2--w...,34,"[2219, 1511, 1515, 1516, 1517, 1518, 1537, 179...",7
7,gas:1--gas:2--heat-exchanger:1--heating-circui...,26,"[510, 1773, 1777, 1778, 1528, 1529, 1530, 1539...",8
8,district-heating:1--heating-circuit:1,22,"[762, 766, 777, 712, 450, 389, 391, 1725, 1726...",9
9,district-heating:1--heating-circuit:1--heating...,22,"[600, 1488, 1724, 796, 1772, 1385, 1386, 1389,...",10


In [9]:
# Sum the number of occurrences for top 10 and top 20
top_10_sum = grouped.head(10)['Occurrence'].sum()
top_20_sum = grouped.head(20)['Occurrence'].sum()
print(f"total LayoutID: {df.LayoutID.count()}")
print(f"top 10 LayoutIDs count: {top_10_sum}")
print(f"top 20 LayoutIDs count: {top_20_sum}")

print(f"Coverage top 10 LayoutIDs: {100*top_10_sum/df.LayoutID.count(): .1f} %")
print(f"Coverage top 20 LayoutIDs: {100*top_20_sum/df.LayoutID.count(): .1f} %")

total LayoutID: 1138
top 10 LayoutIDs count: 525
top 20 LayoutIDs count: 657
Coverage top 10 LayoutIDs:  46.1 %
Coverage top 20 LayoutIDs:  57.7 %


# new standard

In [10]:
# preparation add KBns
df = df.merge(grouped[['LayoutID', 'KBn']], on='LayoutID', how='left')
cols = ['KBn'] + [col for col in df.columns if col != 'KBn']
df = df[cols]
df.head()

,KBn,building_id,customerID,customer_name,customer_name,device_type,customer_name,customer_name,address,LayoutID,modular_system
0,46.0,756,66,HwS,HwS,RUT956,HwS,HwS,Wolzogenstr.28,gas:1--gas:2--heating-circuit:1--heating-circu...,"{'name': 'modular-system-entry', 'id': 'modula..."
1,34.0,757,66,HwS,HwS,RUT956,HwS,HwS,Hagelberger Str. 26,gas:1--gas:2--heating-circuit:1,"{'name': 'modular-system-entry', 'id': 'modula..."
2,3.0,758,66,HwS,HwS,RUT956,HwS,HwS,Hochstr. 8,gas:1--heating-circuit:1,"{'name': 'modular-system-entry', 'id': 'modula..."
3,3.0,759,66,HwS,HwS,RUT956,HwS,HwS,Planufer 82a,gas:1--heating-circuit:1,"{'name': 'modular-system-entry', 'id': 'modula..."
4,32.0,761,66,HwS,HwS,RUT956,HwS,HwS,Gabainstr. 13,gas:1--gas:2--global-separation-circuit:1--hea...,"{'name': 'modular-system-entry', 'id': 'modula..."


In [11]:
# Define substrings to remove
remove_parts = ['--heat-exchanger:1', '--global-separation-circuit:1']
# Copy LayoutID and remove unwanted parts
df['LayoutID_Tops'] = df['LayoutID']
# Loop over each part and remove it
for part in remove_parts:
    df['LayoutID_Tops'] = df['LayoutID_Tops'].str.replace(part, '', regex=False)

In [12]:
unique_count = df['LayoutID_Tops'].nunique()
print(f"Number of unique LayoutID values: {unique_count}")

Number of unique LayoutID values: 193


In [13]:
# Group by LayoutID
grouped_new_std = df.groupby('LayoutID_Tops').agg(
    Occurrence=('LayoutID_Tops', 'count'),
    buildingIDs=('building_id', list),
    Variations=('LayoutID', set),
    Variations_KBn=('KBn', set)
).reset_index()

# Add a simple index column starting from 0
grouped_new_std = grouped_new_std.sort_values(by='Occurrence', ascending=False).reset_index(drop=True)
grouped_new_std=grouped_new_std[['LayoutID_Tops','Occurrence','Variations_KBn','buildingIDs','Variations']]

grouped_new_std['Variations_KBn'] = grouped_new_std['Variations_KBn'].apply(
    lambda s: ', '.join(f"KB{int(x)}" for x in sorted(s))
)
grouped_new_std.head(20)

In [14]:
grouped_new_std.head(20)

,LayoutID_Tops,Occurrence,Variations_KBn,buildingIDs,Variations
0,gas:1--heating-circuit:1--warm-water:1,195,"KB1, KB4, KB15, KB30","[768, 769, 770, 778, 787, 304, 1924, 1925, 192...",{gas:1--global-separation-circuit:1--heating-c...
1,gas:1--heating-circuit:1,132,"KB3, KB6, KB14","[758, 759, 499, 507, 1524, 1525, 1649, 1654, 1...","{gas:1--heat-exchanger:1--heating-circuit:1, g..."
2,district-heating:1--heating-circuit:1--warm-wa...,91,"KB2, KB33","[774, 419, 1909, 1910, 1911, 1912, 1913, 1314,...",{district-heating:1--heating-circuit:1--warm-w...
3,gas:1--heating-circuit:1--heating-circuit:2--w...,51,"KB7, KB23, KB37, KB181","[348, 1775, 1779, 2219, 1511, 1515, 1516, 1517...",{gas:1--heat-exchanger:1--heating-circuit:1--h...
4,local-heating-station:1--heating-circuit:1--wa...,45,KB5,"[509, 790, 1005, 1628, 1629, 1630, 1631, 672, ...",{local-heating-station:1--heating-circuit:1--w...
5,gas:1--gas:2--heating-circuit:1--warm-water:1,40,"KB8, KB19, KB62","[478, 510, 1773, 1777, 1778, 1528, 1529, 1530,...",{gas:1--gas:2--global-separation-circuit:1--he...
6,gas:1--gas:2--heating-circuit:1,31,"KB12, KB29, KB34","[757, 773, 429, 1506, 668, 680, 696, 745, 530,...",{gas:1--gas:2--heat-exchanger:1--heating-circu...
7,district-heating:1--heating-circuit:1--heating...,25,"KB10, KB57","[600, 1488, 2468, 1724, 796, 1772, 1385, 1386,...",{district-heating:1--heating-circuit:1--heatin...
8,gas:1--gas:2--heating-circuit:1--heating-circu...,24,"KB22, KB26, KB50","[1928, 1930, 506, 333, 417, 388, 1641, 414, 79...",{gas:1--gas:2--global-separation-circuit:1--he...
9,district-heating:1--heating-circuit:1,22,KB9,"[762, 766, 777, 712, 450, 389, 391, 1725, 1726...",{district-heating:1--heating-circuit:1}


In [15]:
# Sum the number of occurrences for top 10 and top 20
top_10_sum = grouped_new_std.head(10)['Occurrence'].sum()
top_20_sum = grouped_new_std.head(20)['Occurrence'].sum()
print(f"total LayoutID: {df.LayoutID.count()}")
print(f"top 10 LayoutIDs count: {top_10_sum}")
print(f"top 20 LayoutIDs count: {top_20_sum}")

print(f"Coverage top 10 LayoutIDs: {100*top_10_sum/df.LayoutID.count(): .1f} %")
print(f"Coverage top 20 LayoutIDs: {100*top_20_sum/df.LayoutID.count(): .1f} %")

total LayoutID: 1138
top 10 LayoutIDs count: 656
top 20 LayoutIDs count: 802
Coverage top 10 LayoutIDs:  57.6 %
Coverage top 20 LayoutIDs:  70.5 %


# A new layout Id

In [79]:
if False:
    building_id = 1801
    mod_sys = df.loc[df['building_id'] == building_id, 'modular_system'].iloc[0]
    modular_system = ast.literal_eval(mod_sys)
    pprint(modular_system)

In [77]:
def update_new_layoutID(row):
    layout_id = row['LayoutID']
    modular_system = row['modular_system']
    
    # Skip if layout_id is missing or not a string
    if not isinstance(layout_id, str):
        return layout_id
    
    # Convert modular_system from string to dict if needed
    if isinstance(modular_system, str):
        import ast
        modular_system = ast.literal_eval(modular_system)
    
    suffix_map = {}

    for subsystem in modular_system.get('subSystems', []):
        if subsystem.get('name') == 'heating-circuit':
            hc_id = subsystem.get('id')
            params = subsystem.get('parameters', {})

            has_pump = False
            has_valve = False

            sub_comp = params.get('subComponent', {})

            for key, val in sub_comp.items():
                if 'pump' in key and val.get('hideDisplay', False):
                    has_pump = True
                if 'valve' in key and val.get('hideDisplay', False):
                    has_valve = True

            if not params:
                suffix = ':mixed+pump'
            else:
                if has_pump and not has_valve:
                    suffix = ':mixed+nopump'
                elif has_valve and not has_pump:
                    suffix = ':unmixed+pump'
                elif has_valve and has_pump:
                    suffix = ':unmixed+nopump'
                else:
                    suffix = ':mixed+pump'

            suffix_map[hc_id] = suffix

    parts = layout_id.split('--')
    new_parts = []
    for part in parts:
        if part in suffix_map:
            new_parts.append(part + suffix_map[part])
        else:
            new_parts.append(part)

    return '--'.join(new_parts)

# Then:
df['new_layoutID'] = df.apply(update_new_layoutID, axis=1)
#df.head()

In [86]:
if True: #test single buildings
    building_id = 1593#1809 #1801 #
    mod_sys = df.loc[df['building_id'] == building_id, 'modular_system'].iloc[0]
    modular_system = ast.literal_eval(mod_sys)
    print(df.loc[df['building_id'] == building_id, 'new_layoutID'].iloc[0])
    pprint(modular_system)

gas:1--heating-circuit:1:mixed+pump--heating-circuit:2:mixed+pump--warm-water:1
{'buildingID': 1593,
 'id': 'modular-system--overhoffstr-210',
 'layoutID': 'gas:1--heating-circuit:1--heating-circuit:2--warm-water:1',
 'name': 'modular-system-entry',
 'published': True,
 'publishedAt': '2024-09-11T13:18:01.210Z',
 'subSystems': [{'connections': [{'status': 'hot',
                                  'to': {'direction': 'right',
                                         'id': 'warm-water:1',
                                         'subSystem': {'hydraulicLocationIndex': 1,
                                                       'name': 'warm-water',
                                                       'side': 'bottom'}}},
                                 {'from': {'direction': 'right',
                                           'id': 'warm-water:1',
                                           'subSystem': {'hydraulicLocationIndex': 1,
                                                        

# exports

In [16]:
if True:
    filename1='Building_ID_VS_LayoutID_'+extraced_date+'.csv'
    filename2='LayoutID_occurence_'+extraced_date+'.csv'
    filename3='LayoutID_occurence_new_std_'+extraced_date+'.csv'
    filepath1 = os.path.join('OUTs', filename1)
    filepath2 = os.path.join('OUTs', filename2)
    filepath3 = os.path.join('OUTs', filename3)
    df.to_csv(filepath1, index=False)
    grouped.to_csv(filepath2, index=False)
    grouped_new_std.to_csv(filepath3, index=False)